## Lecture : Stream Template

## Lecture : Relationship Model
- need asymmetrical relationship 
- indexes - A list of indexes to create on the model. These could be fields to index for faster searches or, as in our case, fields to make unique. More information
- join() - A query that references another table or another query.

## Lecture : Follow and Unfollow Button
- exposing IEDs could potentially open you up to some security problems.

**test example **
<pre>
import datetime

from flask import Flask, g, render_template, flash, redirect, url_for
from flask.ext.bcrypt import check_password_hash
from flask.ext.login import LoginManager, login_user, current_user, login_required, logout_user

import forms
import models

app = Flask(__name__)
app.secret_key = 'this is our super secret key. do not share it with anyone!'
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = 'login'


@login_manager.user_loader
def load_user(userid):
    try:
        return models.User.select().where(
            models.User.id == int(userid)
        ).get()
    except models.DoesNotExist:
        return None


@app.before_request
def before_request():
    g.db = models.DATABASE
    g.db.connect()
    g.user = current_user
    

@app.after_request
def after_request(response):
    g.db.close()
    return response


@app.route('/register', methods=('GET', 'POST'))
def register():
    form = forms.SignUpInForm()
    if form.validate_on_submit():
        models.User.new(
            email=form.email.data,
            password=form.password.data
        )
        flash("Thanks for registering!") 
    return render_template('register.html', form=form)
  

@app.route('/login', methods=('GET', 'POST'))
def login():
    form = forms.SignUpInForm()
    if form.validate_on_submit():
        try:
            user = models.User.get(
                models.User.email == form.email.data
            )
            if check_password_hash(user.password, form.password.data):
                login_user(user)
                flash("You're now logged in!")
            else:
                flash("No user with that email/password combo")
        except models.DoesNotExist:
              flash("No user with that email/password combo")
    return render_template('register.html', form=form)

@app.route('/secret')
@login_required
def secret():
    return "I should only be visible to logged-in users"

@app.route('/logout')
def logout():
    logout_user()
    return redirect(url_for('login'))
  

@app.route('/')
def index():
    return render_template('index.html')


@app.route('/order', methods=('GET', 'POST'))
def order_lunch():
    form = forms.LunchOrderForm()
    if form.validate_on_submit():
        models.LunchOrder.create(
            user=g.user._get_current_object(),
            date=form.date.data,
            order=form.order.data.strip()
        )
    return render_template('lunch.html', form=form)


@app.route('/today')
@login_required
def today():
    order = models.LunchOrder.select().where(
        models.LunchOrder.date == datetime.date.today() &
        models.LunchOrder.user == g.user._get_current_object()
    ).get()
    return render_template('today.html', order=order)
  

@app.route('/cancel_order/<int:order_id>')
@login_required
def cancel_order(order_id):
    try:
        order = models.LunchOrder.select().where(
            id=order_id,
            user=g.user._get_current_object()
        ).get()
    except models.DoesNotExist:
        pass
    else:
        order.delete_instance()
    return redirect(url_for('index'))


@app.route('/follow/<int:user_id>')
@login_required
def follow(user_id):
    try: 
        to_user = models.User.get(models.User.id == user_id)
    except models.DoesNotExist:
        pass
    else:
        try:
            models.Relationship.create(
                from_user=g.user._get_current_object(),
                to_user=to_user
            )
        except models.INtegrityError:
            pass
        else:
            flash("You're now following {}!".format(to_user.id), "success")
    return redirect(url_for('index'))
    
    
</pre>

## Lecture : Show Follower Posts

## Lecture : Handling 404s

## Lecture : Suggestions
- sysops ? 
- 

* Allow users to upload an avatar
* Allow users to delete their account
* Add a method for blocking users so you don't see their posts
* Add a method for notifying users if they're mentioned in a post
* Add a new leveer to your users, groups. Users belong to a group and you subscribe to a group's updates instead of a single user

# Flask app making project

1. A user model with an e-mail address and a password
1.1 The password should be hashed with bcrypt
2. Views for signing up, loggin in, and loggin out
3. A model for a taco
3.1 The taco should have a user, a protein choice, a cheese choice, a shell choice, and a free-form text field for further options
4. The ability for users to browse other users recent taco creations on the home page